In [ ]:
import os
os.chdir("../..")

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
filename = "data/testMutPerHost.h5"
file = h5py.File(filename)
all_data = {
	name : [simulation["mutationsperhost"][:] for simulation in group.values()]
	for name,group in file.items()
}
file.close()

In [ ]:
survived = {
	name : [data for data in group if data.shape[0] > 50]
	for name,group in all_data.items()
}

In [ ]:
averaged = {
	name : [np.mean(data) for data in group]
	for name,group in survived.items()
}

In [ ]:
nrows = 1 + len(averaged) // 3
fig, axs = plt.subplots(nrows, 3, figsize=(10,3*nrows))
axs = axs.flatten()
for i,(name,data) in enumerate(averaged.items()):
	axs[i].hist(data, alpha=0.7, edgecolor="k")
	axs[i].set_title(name)
fig.tight_layout()

In [ ]:
thisdata = survived["0"]
nrows = 1 + len(thisdata) // 5
fig, axs = plt.subplots(nrows, 5, figsize=(15,3*nrows), sharex=True, sharey=True)
axs = axs.flatten()
for i,data in enumerate(thisdata):
	axs[i].hist(data, bins=np.arange(-0.5,10), alpha=0.7, edgecolor="k")
	axs[i].set_yscale("log")
fig.tight_layout()

In [ ]:
thisdata = survived["1"]
nrows = 1 + len(thisdata) // 5
fig, axs = plt.subplots(nrows, 5, figsize=(15,3*nrows), sharex=True, sharey=True)
axs = axs.flatten()
for i,data in enumerate(thisdata):
	axs[i].hist(data, bins=np.arange(-0.5,10), alpha=0.7, edgecolor="k")
	axs[i].set_yscale("log")
fig.tight_layout()

In [ ]:
thisdata = survived["2"]
nrows = 1 + len(thisdata) // 5
fig, axs = plt.subplots(nrows, 5, figsize=(15,3*nrows), sharex=True, sharey=True)
axs = axs.flatten()
for i,data in enumerate(thisdata):
	axs[i].hist(data, bins=np.arange(-0.5,10), alpha=0.7, edgecolor="k")
	axs[i].set_yscale("log")
fig.tight_layout()

In [ ]:
from scipy.stats import erlang, expon, gamma
from scipy.special import factorial
from scipy.integrate import quad

In [ ]:
epsilon = 0.1
mu = 0.02
k = 3
lambdaE = k * epsilon
lambdaI = k * mu
nrand = int(1e5)
def f(x):
	return erlang.pdf(x, k, scale=1/lambdaE)
def g(x):
	return erlang.pdf(x, k, scale=1/lambdaI)
class ViralPeriodDistribution:
	def __init__(self):
		self.integrand = lambda x,z: (z-x)**(k-1) * x**(k-1) * np.exp(-(lambdaE-lambdaI)*x)
		self.function = lambda z: np.exp(-lambdaI*z) * quad(self.integrand, 0, z, args=(z,))[0]
		self.zmax = 10 / min(lambdaE,lambdaI)
		self.Z = quad(self.function, 0, self.zmax)[0]
	def __call__(self, z):
		if isinstance(z, np.ndarray):
			values = np.empty_like(z)
			for i,el in enumerate(z):
				values[i] = 1 / self.Z * self.function(el)
		else:
			values = 1 / self.Z * self.function(z)
		return values
h = ViralPeriodDistribution()

mr = 0.02
kmut = 10
theta = 1 / kmut / mr
def p(x, n=1):
	return gamma.pdf(x, n*kmut, scale=theta)

In [ ]:
otherfile = h5py.File("data/mutPerHostWithEvents_0_seed-00000.h5")
mutations = otherfile["0/seed-00000/mutationsperhost"][:]
events = otherfile["0/seed-00000/events"][:]
otherfile.close()

In [ ]:
df = pd.DataFrame.from_records(events)
df2 = df.loc[df["newstate"]==73].copy()
df2["newstate"] = 82
df3 = df.loc[df["newstate"]!=73].copy()
df3["newstate"] = 99
df["newstate"] = df["newstate"].replace(to_replace=73, value=69)
df = pd.concat([df,df2,df3])
df["diff"] = df.groupby(["newstate","loc","ID"]).diff().abs()
df.dropna(inplace=True)
df.drop(columns=["t","loc","ID"], inplace=True)
df

In [ ]:
Es = df.loc[df["newstate"]==69]["diff"].to_numpy()
Is = df.loc[df["newstate"]==82]["diff"].to_numpy()
Vs = df.loc[df["newstate"]==99]["diff"].to_numpy()
xmax = 500
xs = np.linspace(0,xmax,1001)
bins = np.linspace(0,xmax,101)

plt.hist(Es, bins=bins, density=True, color="orange", alpha=0.5)
plt.hist(Is, bins=bins, density=True, color="red", alpha=0.5)
plt.hist(Vs, bins=bins, density=True, color="grey", alpha=0.5)
plt.plot(xs, f(xs), color="orange")
plt.plot(xs, g(xs), color="red")
plt.plot(xs, h(xs), color="grey")
plt.plot(xs, p(xs,6), color="blue")

In [ ]:
df.groupby("newstate").mean()

In [ ]:
Is.mean(), Es.mean()

In [ ]:
from TravelAndMutate.haplotypes import Haplotypes
from TravelAndMutate.randominterface import NumpyRandomGenerator

In [ ]:
rng = NumpyRandomGenerator()
dealer = Haplotypes(rng.cpprng, mr)
values = np.empty(int(1e6))
for i in range(values.shape[0]):
	values[i] = dealer.exportMutPeriod()

In [ ]:
plt.hist(values, bins=np.arange(-0.5,140), density=True, alpha=0.5)
plt.plot(xs, p(xs), color="blue")
plt.xlim([0,120])

In [ ]:
class Anticipation:
	def __init__(self, n=1):
		self.viral = ViralPeriodDistribution()
		self.integrand = lambda w,t: self.viral(t+w) * p(w, n)
		self.function = lambda t: quad(self.integrand, max(0,-t), np.inf, args=(t,))[0]
		self.tmax = 1000#50 / min(lambdaE,lambdaI)
		self.Z = quad(self.function, -self.tmax, self.tmax)[0]
	def __call__(self, t):
		if isinstance(t, np.ndarray):
			values = np.empty_like(t)
			for i,el in enumerate(t):
				values[i] = 1 / self.Z * self.function(el)
		else:
			values = 1 / self.Z * self.function(t)
		return values

In [ ]:
ant = Anticipation()

In [ ]:
newxs = np.linspace(-100,150,101)
Vs_sim = df.loc[df["newstate"]==99]["diff"].to_numpy()
values = np.empty_like(Vs_sim)
for i in range(values.shape[0]):
	values[i] = dealer.exportMutPeriod()
ts = Vs_sim - values
plt.plot(newxs, ant(newxs))
plt.hist(ts, bins=np.linspace(-100,150,51), density=True, alpha=0.5)
print()

In [ ]:
quad(ant, -500, 200)

In [ ]:
quad(ant, 0, 200)

In [ ]:
(ts>0).mean()

In [ ]:
np.mean([(arr>0).mean() for arr in survived["2"]])

In [ ]:
values = np.empty(10)
for i in range(values.shape[0]):
	antk = Anticipation(i+1)
	values[i] = quad(antk, 0, 200)[0]

In [ ]:
values

In [ ]:
corrected = np.zeros(values.shape[0]+1)
for i in range(values.shape[0]):
	if i == 0:
		corrected[i] = 1 - values[i]
	else:
		corrected[i] = values[i-1] - values[i]

In [ ]:
corrected

In [ ]:
(corrected * np.arange(corrected.shape[0])).sum()

In [ ]:
np.mean([arr.mean() for arr in survived["2"]])

In [ ]:
plt.plot(np.arange(11), corrected)
plt.yscale("log")
temp,_,_ = plt.hist(survived["2"][1], bins=np.arange(-0.5,11), density=True, alpha=0.5, edgecolor="k")

In [ ]:
(temp * np.arange(temp.shape[0])).sum()

In [ ]:
corrected-temp

In [ ]:
(corrected-temp)[0], (corrected-temp)[1:].sum()